In [1]:
import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

     Cloning git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`
    Updating git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`
    Updating registry at `C:\Users\kart-\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `C:\Users\kart-\.julia\environments\v1.6\Project.toml`
  [18b7da76] + JuliaAcademyData v0.1.0 `https://github.com/JuliaComputing/JuliaAcademyData.jl#main`
    Updating `C:\Users\kart-\.julia\environments\v1.6\Manifest.toml`
  [18b7da76] + JuliaAcademyData v0.1.0 `https://github.com/JuliaComputing/JuliaAcademyData.jl#main`
Precompiling project...
  ✓ JuliaAcademyData
  1 dependency successfully precompiled in 3 seconds (92 already precompiled)
  Activating environment at `C:\Users\kart-\.julia\packages\JuliaAcademyData\3C2GY\courses\Parallel_Computing\Project.toml`
   Installed ArrayInterface ───────── v2.2.0
   Installed Reexport ─────────────── v0.2.

# Multithreading

Now we're finally ready to start talking about running things on multiple
processors! Most computers (even cell phones) these days have multiple cores
or processors — so the obvious place to start working with parallelism is
making use of those from within our Julia process.

The first challenge, though, is knowing precisely how many "processors" you have.
"Processors" is in scare quotes because, well, it's complicated.

In [ ]:
versioninfo(verbose = true)

In [4]:
# ;cat /proc/cpuinfo # on Linux machines

using Hwloc
Hwloc.num_physical_cores()

6

What your computer reports as the number of processors might not be the same
as the total number of "cores". While sometimes virtual processors can add
performance, parallelizing a typical numerical computation over these virtual
processors will lead to significantly worse performance because they still
have to share much of the nuts and bolts of the computation hardware.

Julia is somewhat multithreaded by default! BLAS calls (like matrix multiplication) are
already threaded:

In [5]:
using BenchmarkTools
A = rand(2000, 2000);
B = rand(2000, 2000);
@btime $A*$B;

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1342


  81.959 ms (2 allocations: 30.52 MiB)


This is — by default — already using all your CPU cores! You can see the effect
by changing the number of threads (which BLAS supports doing dynamically):

In [6]:
using LinearAlgebra
BLAS.set_num_threads(1)
@btime $A*$B
BLAS.set_num_threads(4)
@btime $A*$B

  281.080 ms (2 allocations: 30.52 MiB)
  103.533 ms (2 allocations: 30.52 MiB)


2000×2000 Matrix{Float64}:
 506.782  502.772  505.263  498.195  …  504.752  490.158  493.711  495.897
 519.701  511.185  525.351  518.236     524.026  510.359  508.171  514.803
 500.737  493.335  491.528  496.667     496.745  487.804  488.734  486.753
 508.342  503.622  519.13   501.521     508.156  496.494  491.871  497.413
 506.386  499.859  504.492  493.573     499.404  491.191  491.92   497.941
 506.547  498.667  510.92   502.11   …  504.466  492.785  493.748  495.487
 526.076  522.079  521.487  514.272     517.46   507.725  503.569  510.609
 506.637  510.664  517.562  505.809     504.578  500.122  497.75   497.964
 500.934  501.14   506.628  497.119     500.818  491.313  495.665  499.44
 495.333  482.678  494.667  488.883     490.834  478.53   487.437  493.619
 500.238  494.678  503.295  494.821  …  502.669  491.462  489.396  499.157
 501.896  497.607  509.867  496.564     502.774  492.011  484.884  488.658
 508.102  509.826  509.513  510.778     511.124  494.957  501.623  500.455

## What does it look like to implement your _own_ threaded algorithm?

Multithreading support is marked as "experimental" for Julia 1.0 and is
pending a big revamp for Julia version 1.2 or 1.3. The core tenets will be
the same, but it should become much easier to use efficiently.

In [1]:
using .Threads

nthreads()

1

Julia currently needs to start up knowing that it has threading support enabled.

You do that with a environment variable. To get four threads, start Julia with:

```
JULIA_NUM_THREADS=4 julia
```

On JuliaBox, this is a challenge — we don't have access to the launching process!

In [ ]:
;env JULIA_NUM_THREADS=4 julia -E 'using .Threads; nthreads()'

In [7]:
using .Threads
Threads.nthreads()

1

In [13]:
threadid()

1

So we're currently on thread 1. Of course a loop like this will
just set the first element to one a number of times:

In [8]:
A = Array{Union{Int,Missing}}(missing, nthreads())
for i in 1:nthreads()
    A[threadid()] = threadid()
end
A

1-element Vector{Union{Missing, Int64}}:
 1

But if we prefix it with `@threads` then the loop body runs on all threads!

In [9]:
@threads for i in 1:nthreads()
    A[threadid()] = threadid()
end
A

1-element Vector{Union{Missing, Int64}}:
 1

So let's try implementing our first simple threaded algorithm — `sum`:

In [10]:
function threaded_sum1(A)
    r = zero(eltype(A))
    @threads for i in eachindex(A)
        @inbounds r += A[i]
    end
    return r
end

A = rand(10_000_000)
threaded_sum1(A)
@time threaded_sum1(A)

  0.248875 seconds (20.00 M allocations: 305.176 MiB, 6.79% gc time)


4.999846923532342e6

In [11]:
sum(A)
@time sum(A)

  0.004362 seconds (1 allocation: 16 bytes)


4.999846923532457e6

Whoa! What happened? Not only did we get the wrong answer, it was _slow_ to get it!

In [12]:
function threaded_sum2(A)
    r = Atomic{eltype(A)}(zero(eltype(A)))
    @threads for i in eachindex(A)
        @inbounds atomic_add!(r, A[i])
    end
    return r[]
end

threaded_sum2(A)
@time threaded_sum2(A)

  0.070153 seconds (9 allocations: 976 bytes)


4.999846923532342e6

Alright! Now we got the correct answer (modulo some floating point associativity),
but it's still slower than just doing the simple thing on 1 core.

In [ ]:
threaded_sum2(A) ≈ sum(A)

But it's still slow! Using atomics is much slower than just adding integers
because we constantly have to go and check _which_ processor has the latest
work! Also remember that each thread is running on its own processor — and
that processor also supports SIMD!  Well, that is if it didn't need to worry
about syncing up with the other processors...

In [13]:
function threaded_sum3(A)
    r = Atomic{eltype(A)}(zero(eltype(A)))
    len, rem = divrem(length(A), nthreads())
    @threads for t in 1:nthreads()
        rₜ = zero(eltype(A))
        @simd for i in (1:len) .+ (t-1)*len
            @inbounds rₜ += A[i]
        end
        atomic_add!(r, rₜ)
    end
    # catch up any stragglers
    result = r[]
    @simd for i in length(A)-rem+1:length(A)
        @inbounds result += A[i]
    end
    return result
end

threaded_sum3(A)
@time threaded_sum3(A)

  0.004077 seconds (8 allocations: 656 bytes)


4.999846923532452e6

Dang, that's complicated. There's also a problem:

In [14]:
threaded_sum3(rand(10) .+ rand(10)im) # try an array of complex numbers!

LoadError: TypeError: in Atomic, in T, expected T<:Union{Bool, Float16, Float32, Float64, Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}, got Type{ComplexF64}

Isn't there an easier way?

In [ ]:
R = zeros(eltype(A), nthreads())

In [ ]:
function threaded_sum4(A)
    R = zeros(eltype(A), nthreads())
    @threads for i in eachindex(A)
        @inbounds R[threadid()] += A[i]
    end
    r = zero(eltype(A))
    # sum the partial results from each thread
    for i in eachindex(R)
        @inbounds r += R[i]
    end
    return r
end

threaded_sum4(A)
@time threaded_sum4(A)

This sacrifices our ability to `@simd` so it's a little slower, but at least we don't need to worry
about all those indices! And we also don't need to worry about atomics and
can again support arrays of any elements:

In [ ]:
threaded_sum4(rand(10) .+ rand(10)im)

## Key takeaways from `threaded_sum`:

* Beware shared state across threads — it may lead to wrong answers!
    * Protect yourself by using atomics (or [locks/mutexes](https://docs.julialang.org/en/v1/base/multi-threading/#Synchronization-Primitives-1))
    * Better yet: divide up the work manually such that the inner loops don't
      share state. `@threads for i in 1:nthreads()` is a handy idiom.
    * Alternatively, just use an array and only access a single thread's elements

# Beware of global state (even if it's not obvious!)

Another class of algorithm that you may want to parallelize is a monte-carlo
problem. Since each iteration is a new random draw, and since you're interested
in looking at the aggregate result, this seems like it should lend itself to
parallelism quite nicely!

In [ ]:
using BenchmarkTools

In [ ]:
function serialpi(n)
    inside = 0
    for i in 1:n
        x, y = rand(), rand()
        inside += (x^2 + y^2 <= 1)
    end
    return 4 * inside / n
end
serialpi(1)
@time serialpi(100_000_000)

In [ ]:
using .Threads
nthreads()

Let's use the techniques we learned above to make a fast threaded implementation:

In [ ]:
function threadedpi(n)
    inside = zeros(Int, nthreads())
    @threads for i in 1:n
        x, y = rand(), rand()
        @inbounds inside[threadid()] += (x^2 + y^2 <= 1)
    end
    return 4 * sum(inside) / n
end
threadedpi(100_000_000)
@time threadedpi(100_000_000)

Ok, now why didn't that work?  It's slow! Let's look at the sequence of random
numbers that we generate:

In [ ]:
import Random
Random.seed!(0)
N = 20000
Rserial = zeros(N)
for i in 1:N
    Rserial[i] = rand()
end
Rserial

In [ ]:
Random.seed!(0)
Rthreaded = zeros(N)
@threads for i in 1:N
    Rthreaded[i] = rand()
end
Rthreaded

In [ ]:
Set(Rserial) == Set(Rthreaded)

In [ ]:
indexin(Rserial, Rthreaded)

Aha, `rand()` isn't threadsafe! It's mutating (and reading) some global each
time to figure out what to get next. This leads to slowdowns — and worse — it
skews the generated distribution of random numbers since some are repeated!!

In [ ]:
const ThreadRNG = Vector{Random.MersenneTwister}(undef, nthreads())
@threads for i in 1:nthreads()
    ThreadRNG[Threads.threadid()] = Random.MersenneTwister()
end
function threadedpi2(n)
    inside = zeros(Int, nthreads())
    len, rem = divrem(n, nthreads())
    rem == 0 || error("use a multiple of $(nthreads()), please!")
    @threads for i in 1:nthreads()
        rng = ThreadRNG[threadid()]
        v = 0
        for j in 1:len
            x, y = rand(rng), rand(rng)
            v += (x^2 + y^2 <= 1)
        end
        inside[threadid()] = v
    end
    return 4 * sum(inside) / n
end
threadedpi2(10)
@time threadedpi2(100_000_000)

As an aside, be careful about initializing many `MersenneTwister`s with
different states. Better to use [`randjump`](https://docs.julialang.org/en/v1/manual/parallel-computing/#Side-effects-and-mutable-function-arguments-1) to skip ahead for a single state.

# Beware oversubscription

Remember how BLAS is threaded by default? What happens if we try to `@threads`
something that uses BLAS?

In [ ]:
Ms = [rand(1000, 1000) for _ in 1:100]
function serial_matmul(As)
    first_idxs = zeros(length(As))
    for i in eachindex(As)
        @inbounds first_idxs[i] = (As[i]'*As[i])[1]
    end
    first_idxs
end
serial_matmul(Ms);
@time serial_matmul(Ms);

In [ ]:
using LinearAlgebra
BLAS.set_num_threads(nthreads()) # Explicitly tell BLAS to use the same number of threads
function threaded_matmul(As)
    first_idxs = zeros(length(As))
    @threads for i in eachindex(As)
        @inbounds first_idxs[i] = (As[i]'*As[i])[1]
    end
    first_idxs
end
threaded_matmul(Ms)
@time threaded_matmul(Ms);

In [ ]:
BLAS.set_num_threads(1)
@time threaded_matmul(Ms);

In [ ]:
@time serial_matmul(Ms) # Again, now that BLAS has just 1 thread

# Beware "false sharing"

Remember the memory latency table?


| System Event                   | Actual Latency | Scaled Latency |                          |
| ------------------------------ | -------------- | -------------- | ------------------------ |
| One CPU cycle                  |     0.4 ns     |     1 s        | ← work happens here     |
| Level 1 cache access           |     0.9 ns     |     2 s        |                          |
| Level 2 cache access           |     2.8 ns     |     7 s        |                          |
| Level 3 cache access           |      28 ns     |     1 min      |                          |
| Main memory access (DDR DIMM)  |    ~100 ns     |     4 min      | ← we have control here  |

This is what a typical modern cpu looks like:

![Intel Core i7](https://raw.githubusercontent.com/JuliaComputing/JuliaAcademyData.jl/master/courses/Parallel_Computing/images/i7.jpg)

Multiple cores on the same processor share the L3 cache, but do not share L1 and L2 caches! So what happens if we're accessing and mutating data from the same array across multiple cores?

![Cache coherency](https://raw.githubusercontent.com/JuliaComputing/JuliaAcademyData.jl/master/courses/Parallel_Computing/images/false-sharing.gif)

Unlike "true" sharing — which we saw above — false sharing will still return the correct answer! But it does so at the cost of performance. The cores recognize they don't have exclusive access to the cache line and so upon modification they alert all other cores to invalidate and re-fetch the data.

```julia
function threaded_sum4(A)
    R = zeros(eltype(A), nthreads())
    @threads for i in eachindex(A)
        @inbounds R[threadid()] += A[i]
    end
    r = zero(eltype(A))
    # sum the partial results from each thread
    for i in eachindex(R)
        @inbounds r += R[i]
    end
    return r
end
```

## Further improvements coming here!

PARTR — the threading improvement I discussed at the beginning aims to address
this problem of having library functions implemented with `@threads` and then
having callers call them with `@threads`. Uses a state-of-the-art work queue
mechanism to make sure that all threads stay busy.

# Threading takeaways:

* It's easy! Just start Julia with `JULIA_NUM_THREADS` and tack a `@threads` on your loop
* Well, not so fast
    * Be aware of your hardware to set `JULIA_NUM_THREADS` appropiately
    * Beware shared state (for both performance and correctness)
    * Beware global state (even if it's not obvious)
    * Beware false sharing (if Julia/LLVM don't handle it for you)
* We need to think carefully about how to design parallel algorithms!